In [1]:
!pip install selenium
!pip install beautifulsoup4
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

     |████████████████████████████████| 911kB 25.9MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64

In [2]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re

In [3]:
def get_all_product_link_per_page(soup):
    link_each_product_per_page = []
    link_each_product_div = soup.find_all('div',{'class':'col-xs-2-4 shopee-search-item-result__item'})
    for link_each_product_field in link_each_product_div:
        link_each_product = link_each_product_field.find_all('a', {'data-sqe': 'link'})
        link_each_product = ['https://shopee.co.id'+each_link.get('href') for each_link in link_each_product]
        link_each_product_per_page.extend(link_each_product)
    return link_each_product_per_page

In [8]:
def shopee_product_link_scraper(ref_link, defined_total_page=5):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
    driver.get(ref_link)
    time.sleep(2)  # Allow 2 seconds for the web page to open
    scroll_pause_time = 1 
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
 
    data_result_dict = {'Product Link': []}

    for page in range(defined_total_page):
        i = 1
        while True:
            driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
            i += 1
            time.sleep(scroll_pause_time)
            scroll_height = driver.execute_script("return document.body.scrollHeight;")  
            if (screen_height) * i > scroll_height:
                break 

        time.sleep(10)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        all_product_link_per_page = get_all_product_link_per_page(soup)
        print("[INFO] page: {}; recorded product num: {}".format(page+1, len(all_product_link_per_page)))
        data_result_dict['Product Link'].extend(all_product_link_per_page)
        next_button = driver.find_element_by_xpath('//button[@class="shopee-icon-button shopee-icon-button--right "]')
        webdriver.ActionChains(driver).move_to_element(next_button).click(next_button).perform()
        time.sleep(10)
    
    data_result_df = pd.DataFrame.from_dict(data_result_dict)
    data_result_df.to_csv('shopee_link_data.csv', index=False)
    return data_result_df

In [9]:
ref_link = 'https://shopee.co.id/Handphone-Tablet-cat.40.1211?locations=Jabodetabek%2CDKI%2520Jakarta&page=0'
tokped_data = shopee_product_link_scraper(ref_link, defined_total_page=2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


[INFO] page: 1; recorded product num: 36
[INFO] page: 2; recorded product num: 36
